In [1]:
import torch
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
import pymysql
from flask import Flask # 플라스크 클래스 임포트
from flask import request, redirect
import cv2 
import pickle
import sys
import os.path
import datetime
from os import rename, listdir
import shutil
import time

In [2]:
def Circuit_cnt(label_list):
    dic =  { 
             'AC_Source' : 0, 
             'BJT':1 ,
             'Battery':2,
             'Capacitor':3,
             'Current_Source':4,
             'DC_Source':5,
             'Diode':6,
             'Ground':7,
             'Inductor':8,
             'MOSFET':9,
             'Resistors':10, 
             'Voltage_Source':11,
           }
    # 딕셔너리의 key값들만 리스트로.
    parts=list(dic.keys())
    # 딕셔너리 값 0으로.
    dic3=dic.fromkeys(dic, 0) #fromKeys : dictionary 의 values 값을 0으로 바꿔주는 내장함수 
    
    # dic의 각 부품의 인덱스값과 label_list의 값이 같을 경우 해당되는 부품의 인덱스에 카운트해주기 위해 dic3에 카운트해줄것.
    for i in parts:
        for j in label_list:
            if dic[i]==j:
                dic3[i]=dic3[i]+1
        
    # 나온 부품들 DB에 넣을 코드를 작성하는 공간           
    return dic3

In [3]:
def Board_conn(member_name, circuit_circuit, now): # 회원이름 / 욜로돌린 사진 / 저항 이름 / 저항 개수 파라미터
    # database에 접근
    db = pymysql.connect(
                         host='project-db-stu.ddns.net',port=3307,user='hr',passwd='hr',db='hr',charset='utf8'
                        )

    # database를 사용하기 위한 cursor를 세팅
    cursor = db.cursor()
    
    cursor.execute(
    """INSERT INTO 
        board_list (
            mem_id,
            circuit_img,
            board_time
          )
    VALUES (%s,%s,%s)""", (member_name, circuit_circuit, now))
    
    db.commit() # insert 쿼리문은 commit 해줘야 데이터가 들어감 
    return "success" 

#Board_conn(member_name, circuit_circuit,now)

In [ ]:
cnt = 1

app = Flask(__name__)

@app.route('/', methods = ["GET","POST"])
def FromJsptoFlask():
    if request.method == "POST":
        # jsp로부터 온 값
        global cnt
        img_data = request.form["test"] # JSP-Ajax에서 보내준 사진 이름 ex) mokoko.png 
        
        ########################################################################################################################
        member_name = request.form["member_name"] # JSP-Ajax로 들어온 값 ex) ysu  # 데이터베이스에 넣을 값
        print("jsp로부터 들어온 회원 아이디 :"+member_name)
        ########################################################################################################################
        
        # 커스텀 yolov5 모델 생성 
        #model = torch.hub.load('ultralytics/yolov5', 'custom', path='best_lcg.pt', force_reload=True) # force_reload = True => 캐쉬삭제 및 최적화
        ########################################################################################################################
        #model = torch.hub.load('ultralytics/yolov5', 'custom', path='ksm_circuit19_v1.pt', force_reload=True)
        model = torch.hub.load('ultralytics/yolov5', 'custom', path='0929_lcg_circuit12-v2.1_x2x2.pt', force_reload=True)
        #model = torch.hub.load('ultralytics/yolov5', 'custom', path='best_lhj', force_reload=True)
        ########################################################################################################################
        # label_img : 라벨링된 회로도 이미지
        img_data_file = "C:/Users/user/Circuit_team_project/Circuit_team/src/main/webapp/Circuit_images/"+str(img_data)
        
        label_img = model(img_data_file)# 커스텀 모델에 사진을 집어넣음
        label_img.pred
        change_dimension = label_img.pred[0]
        # 예측(부품예상) 값을 담는 리스트
        label_list = []
        print("이클립스 저장 이미지 : ", img_data_file)
        print("욜로 결과 이미지 :",label_img)
        for i in range(len(change_dimension)):
            label_list.append(int(change_dimension[i][-1]))
        
        print("label_list : ",label_list) #  입력한 회로도 사진을 yolo로 돌리고 나온 라벨값들을 저장
        print("label_list type :",type(label_list))
        print("label_img type :",type(label_img))
        print('fff', label_img)
        Circuit_cnt(label_list) # 라벨된 회로도 사진을 넣으면 들어있는 부품의 개수를 리턴함
        print(Circuit_cnt(label_list)) # Circuit_cnt 함수가 제대로 실행되는지 확인용
        ######################################################################################################################## 
        
        
        label_img.save(save_dir = "C:/Users/user/Circuit_team_project/Circuit_team/src/main/webapp/Yolo_Circuit_images")
        
        time.sleep(0.2) # 처음 0.2
        shutil.move('C:/Users/user/Circuit_team_project/Circuit_team/src/main/webapp/Yolo_Circuit_images2/'+img_data,
          'C:/Users/user/Circuit_team_project/Circuit_team/src/main/webapp/Yolo_Circuit_images/'+img_data)
                       
        time.sleep(0.2) # 처음 0.2   
        shutil.rmtree('C:/Users/user/Circuit_team_project/Circuit_team/src/main/webapp/Yolo_Circuit_images2')
        
        
        ########################################################################################################################
        # JSP -> FLASK에서 넘어온 데이터 회원의 이름 // 욜로로 돌려진 회로도 사진 //FLASK에서 분류되어진 부품의 이름과 개수를 데이터베이스에 저장
        member_name = member_name # 로그인한 회원 아이디가 들어가짐
        circuit_circuit = img_data # 원본 회로도 사진 파일 이름이 들어가짐
        ########################################################################################################################
        now = datetime.datetime.now()
        ########################################################################################################################
        # if문 넣어서 부품별 구분할까?  // 아 근대 이건 게시판이라서 부품의 개수까지 보여줄 필요가 있나?
        ########################################################################################################################
        Ac_Source_num= Circuit_cnt(label_list)['AC_Source'] # 테스트로 욜로 결과 회로도의 일부 부품인 저항 개수만 일단 테스트
        BJT_num= Circuit_cnt(label_list)['BJT']
        Battery_num= Circuit_cnt(label_list)['Battery']
        Capacitor_num= Circuit_cnt(label_list)['Capacitor']
        Current_Source_num= Circuit_cnt(label_list)['Current_Source']
        DC_Source_num= Circuit_cnt(label_list)['DC_Source']   
        Diode_num= Circuit_cnt(label_list)['Diode']
        Ground_num= Circuit_cnt(label_list)['Ground']
        Inductor_num= Circuit_cnt(label_list)['Inductor']
        MOSFET_num= Circuit_cnt(label_list)['MOSFET']        
        Resistors_num= Circuit_cnt(label_list)['Resistors']
        Voltage_Source_num= Circuit_cnt(label_list)['Voltage_Source']
       
        def SQL_conn(member_name, circuit_circuit, Ac_Source_num, BJT_num, Battery_num, Capacitor_num, Current_Source_num, DC_Source_num,
                 Diode_num, Ground_num, Inductor_num, MOSFET_num, esistors_num, Voltage_Source_num): # 회원이름 / 욜로돌린 사진 / 저항 이름 / 저항 개수 파라미터
            # database에 접근
            db = pymysql.connect(
                                 host='project-db-stu.ddns.net',port=3307,user='hr',passwd='hr',db='hr',charset='utf8'
                                )

            # database를 사용하기 위한 cursor를 세팅
            cursor = db.cursor()

        ########################################################################################################################
            cursor.execute(
            """INSERT INTO 
                circuit_parts_list_12num (
                    mem_id,
                    circuit_img,
                    AC_Source_num,
                    BJT_num,
                    Battery_num,
                    Capacitor_num,
                    Current_Source_num,
                    DC_Source_num,              
                    Diode_num,
                    Ground_num,
                    Inductor_num,
                    MOSFET_num,                   
                    Resistors_num,                   
                    Voltage_Source_num                   
                  )
            VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)""", 
               (member_name, circuit_circuit, Ac_Source_num, BJT_num, Battery_num, Capacitor_num, Current_Source_num, DC_Source_num,
                 Diode_num, Ground_num, Inductor_num, MOSFET_num, Resistors_num, Voltage_Source_num))

            db.commit() # insert 쿼리문은 commit 해줘야 데이터가 들어감 
            return "success" 
        ########################################################################################################################
        
        SQL_conn(member_name, circuit_circuit, Ac_Source_num, BJT_num, Battery_num, Capacitor_num, Current_Source_num, DC_Source_num,
                 Diode_num, Ground_num, Inductor_num, MOSFET_num, Resistors_num, Voltage_Source_num)
        
        ########################################################################################################################
        Board_conn(member_name, circuit_circuit, now)
        ########################################################################################################################
    return img_data_file

if __name__ == "__main__" : # .py 파일에서 main함수 역할
    app.run() # 5000번 포트로 서버 구동

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


jsp로부터 들어온 회원 아이디 :lcg


Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\user/.cache\torch\hub\master.zip
YOLOv5  2022-10-12 Python-3.8.13 torch-1.8.2+cu111 CPU

Fusing layers... 
YOLOv5s summary: 157 layers, 7042489 parameters, 0 gradients
Adding AutoShape... 
Saved 1 image to C:\Users\user\Circuit_team_project\Circuit_team\src\main\webapp\Yolo_Circuit_images2


이클립스 저장 이미지 :  C:/Users/user/Circuit_team_project/Circuit_team/src/main/webapp/Circuit_images/6.jpg
욜로 결과 이미지 : image 1/1: 508x590 2 BJTs, 2 Capacitors, 2 Diodes, 4 Resistors, 1 Voltage_Source
Speed: 22.9ms pre-process, 197.7ms inference, 15.0ms NMS per image at shape (1, 3, 576, 640)
label_list :  [3, 3, 11, 1, 1, 10, 10, 10, 10, 6, 6]
label_list type : <class 'list'>
label_img type : <class 'models.common.Detections'>
fff image 1/1: 508x590 2 BJTs, 2 Capacitors, 2 Diodes, 4 Resistors, 1 Voltage_Source
Speed: 22.9ms pre-process, 197.7ms inference, 15.0ms NMS per image at shape (1, 3, 576, 640)
{'AC_Source': 0, 'BJT': 2, 'Battery': 0, 'Capacitor': 2, 'Current_Source': 0, 'DC_Source': 0, 'Diode': 2, 'Ground': 0, 'Inductor': 0, 'MOSFET': 0, 'Resistors': 4, 'Voltage_Source': 1}


127.0.0.1 - - [12/Oct/2022 09:21:27] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [12/Oct/2022 09:21:27] "POST / HTTP/1.1" 200 -


In [ ]:
# DB에 저장되어있는 사진에 접근

def SQL_conn(imageId): # 함수 안에 mem_id, circuit_name을 넣어야함!
    # database에 접근
    db = pymysql.connect(
                         host='project-db-stu.ddns.net',port=3307,user='hr',passwd='hr',db='hr',charset='utf8'
                        )

    # database를 사용하기 위한 cursor를 세팅합니다.
    cursor = db.cursor()

    sql = "select imageFileName_test from image_test where imageId = "+str(imageId)+";"

    # SQL query 실행

    cursor.execute(sql)

    result = cursor.fetchall()
    
    DB_img = result[0][0]
    
    img = "C:/Users/smhrd/java_android/Circuit_team/src/main/webapp/Circuit_images"+DB_img 

    # 이건 이제 이미지 파일을 꺼내올려고 하는거임
    return img

In [ ]:
# DB에 있는 사진을 추출해서 Yolo로 돌림 그리고 결과 
def Yolov5(img_path_from_sql):
    import pickle
    import torch
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import datasets, layers, models
    # 커스텀 yolov5 모델 생성 
    model = torch.hub.load('ultralytics/yolov5', 'custom', path='best_lcg.pt', force_reload=True) # force_reload = True => 캐쉬삭제 및 최적화
    # 회로도 사진 
    img = img_path_from_sql
    # label_img : 라벨링된 이미지
    label_img = model(img)# 커스텀 모델에 사진을 집어넣음
    label_img.print() # 해당 사진에 어떤 부품이 있고 몇개가 있는지 출력
    label_img.pred # 라벨링된 부품의 좌표 index = 0~3 , 신뢰성 = index : 4 , 예측(부품예상) 값 = index : 5
    # 2차원 배열 값 -> 1차원 배열 값 전환
    change_dimension = label_img.pred[0]
    # 테스트용 -> 1차원 배열중 index = 1번 값 
    #change_dimension[1]
    # 예측(부품예상) 값을 담는 리스트
    label_list = []
    for i in range(len(change_dimension)):
        label_list.append(int(change_dimension[i][-1]))
    print(label_list)
    # 해당 모델의 학습되어 있는 부품 리스트 
#     label_img.names
    # 라벨링된 사진 출력
    label_img.show()
    return label_list, label_img #label_img.names도 출력나오게 나중에 조정

In [ ]:
import shutil

